In [ ]:
import tkinter as tk
import socket
import cv2
import pickle
import struct
from PIL import Image, ImageTk
import threading
import pyshine as ps

IP = "172.17.0.152"
PORT = 1234
PORT1 = 1235
PORT2 = 1236
PORT3 = 1237
PORT4 = 1233
SIZE = 4096
FORMAT = "utf-8"
DISCONNECT_MSG = "BYE"


class VideoChatUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Chat")
        self.root.geometry("1000x500")  # Adjusted window size
        self.root.configure(bg='#ffff66')  # Light yellow background

        self.create_ui()

        # for video
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((IP, PORT))

        # for recv messages
        self.client_socket1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket1.connect((IP, PORT1))

        # for recev file
        self.client_socket3 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket3.connect((IP, PORT3))

        # for recev audio
        self.client_socket4 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket4.connect((IP, PORT4))

        self.chat_thread = threading.Thread(target=self.receive_messages)
        self.video_thread = threading.Thread(target=self.video_streaming)
        self.file_receive_thread = threading.Thread(target=self.recevive_File)
        self.audio_thread = threading.Thread(target=self.audio_streaming)
        self.main_thread = threading.Thread(target=self.main1)

        self.chat_thread.start()
        self.video_thread.start()
        self.file_receive_thread.start()
        self.audio_thread.start()
        self.main_thread.start()

    def create_ui(self):
        print("hai")
        # Create Canvas widgets for displaying video streams
        self.canvas1 = tk.Canvas(self.root, width=500, height=480)
        self.canvas1.pack(side="left")

        self.canvas2 = tk.Canvas(self.root, width=500, height=480)
        self.canvas2.pack(side="right")

        # Buttons for various functionalities
        chat_button = tk.Button(self.root, text="Start Chat", command=self.start_chat, bg='#85ff89')
        chat_button.pack(side="top", pady=10)

        file_transfer_button = tk.Button(self.root, text="Start File Transfer", command=self.start_file_transfer, bg='#ff8c8c')
        file_transfer_button.pack(side="top", pady=10)

        video_conferencing_button = tk.Button(self.root, text="Start Video Conferencing", command=self.start_video_conferencing, bg='#8c8cff')
        video_conferencing_button.pack(side="top", pady=10)

        exit_button = tk.Button(self.root, text="Exit", command=self.exit_application, bg='#ff9999')
        exit_button.pack(side="bottom", pady=10)

    def start_chat(self):
        chat_window = tk.Toplevel(self.root)
        chat_window.title("Chat Page")
        chat_window.geometry("400x300")
        chat_window.configure(bg='#66ff66')  # Light green background

    def start_file_transfer(self):
        file_transfer_window = tk.Toplevel(self.root)
        file_transfer_window.title("File Transfer Page")
        file_transfer_window.geometry("400x300")
        file_transfer_window.configure(bg='#ff6666')  # Light red background

    def start_video_conferencing(self):
        video_conferencing_window = tk.Toplevel(self.root)
        video_conferencing_window.title("Video Conferencing Page")
        video_conferencing_window.geometry("400x300")
        video_conferencing_window.configure(bg='#6666ff')  # Light blue background

    def exit_application(self):
        self.root.destroy()

    def receive_messages(self):
        while True:
            try:
                msg = client_socket1.recv(SIZE).decode(FORMAT)
                if msg == DISCONNECT_MSG:
                    print("[Server] Disconnected.")
                    break
                print(f"[Client] {msg}")
            except ConnectionAbortedError:
                print("[Server] Connection was aborted by the server.")
                break

    def audio_streaming(self):
        data = b""
        payload_size = struct.calcsize("Q")
        audio, context = ps.audioCapture(mode=mode)
        # ps.showPlot(context1,name1)
        if client_socket4:
            while True:

                try:
                    frame = audio.get()

                    a = pickle.dumps(frame)
                    message = struct.pack("Q", len(a))+a
                    client_socket4.sendall(message)
                except:
                    continue

    def receive_audio(self):
        pass  # Add logic to receive and play audio

    def video_streaming(self):
        pass  # Add logic for video streaming

    def receive_frames(self):
        pass  # Add logic to receive and display video frames

    def send_file(self, client, file_path):
        pass  # Add logic to send a file

    def recevive_File(self):
        pass  # Add logic to receive a file

    def to_chat(self):
        while True:
            msg = input("> ")
            if (msg == "-1"):
                break
            client_socket1.send(msg.encode(FORMAT))

    def to_send_file(self):
        client_socket2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket2.connect((IP, PORT2))
        FILENAME = input(
            "Enter the files name to send with extension (For ex: sample.txt) or for receiving Enter -1: ")
        print(FILENAME)
        client_socket2.send(FILENAME.encode(FORMAT))
        msg = client_socket2.recv(SIZE).decode(FORMAT)
        no = input(msg)
        client_socket2.send(no.encode(FORMAT))
        send_file(client_socket2, FILENAME)
        print("hello")
        client_socket2.close()

    def main1(self):
        while True:

            print("1.To Chat\n 2.To Send File")
            option = int(input("Enter here:"))
            if option == 1:
                to_chat()
            else:
                to_send_file()


if __name__ == "__main__":
    root = tk.Tk()
    ui = VideoChatUI(root)
    root.mainloop()
